### Assignment #4: Basic UI

DS4003 | Spring 2024

Objective: Practice buidling basic UI components in Dash. 

Task: Build an app that contains the following components user the gapminder dataset: `gdp_pcap.csv`. [Info](https://www.gapminder.org/gdp-per-capita/)

UI Components:
A dropdown menu that allows the user to select `country`
-   The dropdown should allow the user to select multiple countries
-   The options should populate from the dataset (not be hard-coded)
A slider that allows the user to select `year`
-   The slider should allow the user to select a range of years
-   The range should be from the minimum year in the dataset to the maximum year in the dataset
A graph that displays the `gdpPercap` for the selected countries over the selected years
-   The graph should display the gdpPercap for each country as a line
-   Each country should have a unique color
-   Graph DOES NOT need to interact with dropdown or slider
-   The graph should have a title and axis labels in reader friendly format  

Layout:  
- Use a stylesheet
- There should be a title at the top of the page
- There should be a description of the data and app below the title (3-5 sentences)
- The dropdown and slider should be side by side above the graph and take up the full width of the page
- The graph should be below the dropdown and slider and take up the full width of the page

Submission: 
- There should be only one app in your submitted work
- Comment your code
- Submit the html file of the notebook save as `DS4003_A4_LastName.html`


**For help you may use the web resources and pandas documentation. No co-pilot or ChatGPT.**

In [1]:
# import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, callback

#import style sheets 
stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] # load the CSS stylesheet

In [2]:
#loading the gapminder data
df = pd.read_csv("gdp_pcap.csv")

#checking data 
print(df.head())

#check all the columns in df
#df.columns.values.tolist()


       country  1800  1801  1802  1803  1804  1805  1806  1807  1808  ...  \
0  Afghanistan   599   599   599   599   599   599   599   599   599  ...   
1       Angola   465   466   469   471   472   475   477   479   481  ...   
2      Albania   585   587   588   590   592   593   595   597   598  ...   
3      Andorra  1710  1710  1710  1720  1720  1720  1730  1730  1730  ...   
4          UAE  1420  1430  1430  1440  1450  1450  1460  1460  1470  ...   

    2091   2092   2093   2094   2095   2096   2097   2098   2099   2100  
0   4800   4910   5030   5150   5270   5390   5520   5650   5780   5920  
1  24.8k  25.3k  25.9k  26.4k  26.9k  27.4k    28k  28.5k  29.1k  29.6k  
2    54k  54.6k  55.2k  55.8k  56.4k  56.9k  57.5k  58.1k  58.7k  59.2k  
3  79.3k  79.5k  79.8k  80.1k  80.4k  80.7k    81k  81.2k  81.5k  81.8k  
4  92.5k  92.6k  92.6k  92.7k  92.8k  92.9k  92.9k    93k  93.1k  93.1k  

[5 rows x 302 columns]


In [3]:
#find how many countries there are
len(df)

195

In [4]:
#get the date and have data melted since it is long across the row --> makes the data long format

melted_df = pd.melt(df, id_vars=['country'], var_name='year', value_name='value')

#check and displays df that has all the countries within a year --> year has separate col
melted_df.head(70)



,country,year,value
0,Afghanistan,1800,599
1,Angola,1800,465
2,Albania,1800,585
3,Andorra,1800,1710
4,UAE,1800,1420
...,...,...,...
65,Gambia,1800,991
66,Guinea-Bissau,1800,613
67,Equatorial Guinea,1800,454
68,Greece,1800,1620


In [5]:
#finding min and max year

min_year = int(melted_df["year"].min())
print(min_year)

max_year = int(melted_df["year"].max())
print(max_year)

print(type(max_year))



1800
2100
<class 'int'>


In [6]:
#getting rid of the Ks in the value column

def val_float(x):
    if type(x) == float or type(x)== int:
        return x #if it is a float or int, then keep as is
    if 'k' in x:
        if len(x) > 1 :
            return float(x.replace('k', '')) * 1000
        return 1000.0


melted_df['value'] = melted_df['value'].apply(val_float)




    

In [7]:
# a graph that displays the the gdpPercap over the selected years
#the graph displays gdpPercap for each country as a line
#each country has its own coolor
#graph should have a title and axis labels 

#building a graph for the selected years = line graph 

fig_line = px.line(melted_df,
                    x = "year",
                    y = "value",
                    color = "country",
                    title = "GDP per Capita by Country from the year 1800 - 2100"
                    )
#change the x/y axis
fig_line.update_layout(
    xaxis_title = "Year",
    yaxis_title = "GDP"
)

fig_line.show()


#graph has a scroll to show all the different countries 

In [19]:
# add rows to layout -->  Div = html; classes = css
#getting rid of the Div

#use a style sheet 
#title at the top 
#description of data and app below the title
#dropdown/slider should be side by side and take up full width
#graph should be below the dropdown and slider and take up full page  

app = Dash(__name__)
app.layout = html.Div([ #parent Div

        html.Div( #div not included in className= "rows"
                dcc.Markdown('''
                    # App for GDP in Countries from Year 1800 - 2100
                    
                    Description: 
                    This is a dashboard that displays the GDP per capita of each country from the year 1800 to 2100. The data tracks 195 countries throughout
                    the years and their respective GDPs. There is a dropdown and slider bar to select the country and years that you are interested in investigating. The data does 
                    include years past the current date, which are meant to serve as estimations of future GDP. 
                    
                    '''
            ),
        ),
        html.Div([ #--> tried to create a parent div that has just slider and dropdown to use className = row
            html.Div(
                dcc.Dropdown(
                    id = 'country dropdown',
                    options = df["country"],
                    value =  ['Angola', 'Albania'], #default is angola and albania (first two values) 
                    multi = True # this makes dropdown multi-select
                ),
            className = "six columns" #takes up half of screen assuming screen = 12 cols 
            ),

            html.Div(
                dcc.RangeSlider(
                    min = min_year, 
                    max = max_year, 
                    step = 1, 
                    value = [1900,1910], #default, note array notation
                    id = 'range-slider-1',
                    marks = {year: str(year) for year in range(1800, 2101, 10)} 
                    #marks is a dict that represents the numerical values and the labels
                    #need to have a mark for everything from 1800 to 2100, thus make a for loop for each mark 
                ), 
                className = "six columns" #half screen
            ), 
        ], className = 'row'),# makes it so all are on the same row
           
        html.Div(
            dcc.Graph(
                figure = fig_line
            ),
        ),
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('country dropdown', 'value'),
     Input('range-slider-1', 'value')])
def update_graph(selected_countries, selected_year):
    filtered_df = melted_df[(df['country'].isin(selected_countries)) & (df['year'] == selected_year)]

    fig = px.line(filtered_df, x='year', y='value', color='country')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0})

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

#runs the app
if __name__ == '__main__':
    app.run()

In [20]:
#getting the callbacks 

#trying with x axis select and y axis fixed
df = melted_df
#get the title onto graph
# fig_line = px.line(df,
#                     x = "year",
#                     y = "value",
#                     color = "country",
#                     title = "GDP per Capita by Country from the year 1800 - 2100"
#                     )
# #change the x/y axis
# fig_line.update_layout(
#     xaxis_title = "Year",
#     yaxis_title = "GDP"
# )

# fig_line.show()

#only need dropdown and slider for interactivity
app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=[{'label': i, 'value': i} for i in df['country'].unique()], #gets all unique vals as options for dropdown
                value=['Afghanistan', 'Angola', 'Albania'], #default 
                id='country-dropdown',
                multi=True
            ),
        ], style={'width': '48%', 'display': 'inline-block'}),
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        min=df['year'].min(),
        max=df['year'].max(),
        id='year-slider',
        step=1,
        value=df['year'].max(),
        marks={str(year): str(year) for year in df['year'].unique()},
    )

])

@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('year-slider', 'value')])
def update_graph(selected_countries, selected_year):
    filtered_df = df[(df['country'].isin(selected_countries)) & (df['year'] == selected_year)] #updates graph based on chosen year and countries

    fig = px.line(filtered_df, x='year', y='value', color='country')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0})

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

#could not get the slider to show 

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

In [ ]:
melted_df.dtypes

country     object
year        object
value      float64
dtype: object